In [9]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 10/07, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
0 | 1
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 1
3b	| 1
3c |	1
3d	| 3
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
7c | 2
8  | 6
**Total** | 72

In [10]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [11]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [12]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
CREATE EXTENSION
COMMENT
SET
CREATE TABLE
psql:<stdin>:49: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:80: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:96: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:112: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:129: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:146: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:178: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:210: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:223: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:251: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:267: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:295: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:322: ERROR:  

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None
imdb,jovyan,UTF8,en_US.utf8,en_US.utf8,None
jovyan,jovyan,UTF8,en_US.utf8,en_US.utf8,None
postgres,jovyan,UTF8,en_US.utf8,en_US.utf8,None
template0,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan
template1,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [13]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [14]:
%%sql
SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


Run the following cell for grading purposes.

In [15]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.

Here are some key things to note:
- There are two cost values: the first is the **startup cost** (cost to return the first row) and the second is the **total cost** (cost to return all rows).
- The unit for the estimated query cost is arbitrary (1 is the cost for reading an 8kB page during a sequential scan)
- When we ask you to identify the query cost, we are looking for the **total cost**
- When we ask you to identify the query time, we are looking for the **execution time** (in ms)
- Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact
- We recognzie that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range

Now, inspect the query plan above, and record the **cost** and **time** for the sample query.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

In [6]:
sample_query_cost = 1193.88
sample_query_timing = 185.896

In [7]:
grader.check("q0")

q0 results: All test cases passed!

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [17]:
%%sql result_1a <<
SELECT namefirst,namelast, a.playerid, yearid 
FROM halloffame AS a, people AS b 
WHERE a.playerid = b.playerid AND a.inducted = 'Y';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable result_1a


In [19]:
result_1a

namefirst,namelast,playerid,yearid
Ty,Cobb,cobbty01,1936
Babe,Ruth,ruthba01,1936
Honus,Wagner,wagneho01,1936
Christy,Mathewson,mathech01,1936
Walter,Johnson,johnswa01,1936
Nap,Lajoie,lajoina01,1937
Tris,Speaker,speaktr01,1937
Cy,Young,youngcy01,1937
Morgan,Bulkeley,bulkemo99,1937
Ban,Johnson,johnsba99,1937


In [20]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [21]:
grader.check("q1a")

q1a results: All test cases passed!

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the **execution time** and **cost** for each query.

In [22]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable provided_query


In [15]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..959.06 rows=323 width=25) (actual time=12.406..13.307 rows=323 loops=1)
   Hash Cond: ((halloffame.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=323 width=13) (actual time=0.026..0.752 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=619.70..619.70 rows=19370 width=21) (actual time=12.137..12.139 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=21) (actual time=0.011..5.912 rows=19370 loops=1)
 Planning Time: 2.045 ms
 Execution Time: 13.433 ms
(10 rows)



In [16]:
inducted_hof_query_str = "SELECT namefirst,namelast, a.playerid, yearid FROM halloffame AS a, people AS b WHERE a.playerid = b.playerid AND a.inducted = 'Y';"
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..959.06 rows=323 width=25) (actual time=11.510..12.276 rows=323 loops=1)
   Hash Cond: ((a.playerid)::text = (b.playerid)::text)
   ->  Seq Scan on halloffame a  (cost=0.00..96.39 rows=323 width=13) (actual time=0.029..0.647 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=619.70..619.70 rows=19370 width=21) (actual time=11.209..11.210 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people b  (cost=0.00..619.70 rows=19370 width=21) (actual time=0.014..5.689 rows=19370 loops=1)
 Planning Time: 1.211 ms
 Execution Time: 12.427 ms
(10 rows)



In [23]:
provided_query_cost = 959.06
provided_query_timing = 13.433
your_query_cost = 959.06
your_query_timing = 12.427

In [24]:
grader.check("q1bi")

q1bi results: All test cases passed!

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [25]:
q1b_part2 = ['A', 'C', 'F']

In [26]:
grader.check("q1bii")

q1bii results: All test cases passed!

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [27]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS
SELECT people.namefirst, people.namelast, halloffame.playerid, schools.schoolid, halloffame.yearid
FROM halloffame 
JOIN people ON halloffame.playerid = people.playerid
JOIN collegeplaying ON halloffame.playerid = collegeplaying.playerid
JOIN schools ON schools.schoolid = collegeplaying.schoolid
WHERE halloffame.inducted = 'Y' AND schools.schoolstate = 'CA'
ORDER BY halloffame.yearid, people.playerid;


SELECT * FROM inducted_hof_ca;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
23 rows affected.
Returning data to local variable result_1c


In [36]:
result_1c

namefirst,namelast,playerid,schoolid,yearid
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,usc,1992
Tom,Seaver,seaveto01,cafrecc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


In [28]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [29]:
grader.check("q1c")

q1c results: All test cases passed!

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [30]:
%%sql with_view <<
SELECT schoolid, yearid, COUNT(*)
FROM inducted_hof_ca
GROUP BY schoolid, yearid
ORDER BY yearid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_view


In [40]:
with_view

schoolid,yearid,count
ucla,1962,1
stmarysca,1971,1
camerri,1990,1
cafrecc,1992,1
usc,1992,1
calpoly,2002,4
sandiegost,2007,3
capasad,2008,1
sandiegost,2010,2
calavco,2011,1


In [31]:
%%sql without_view <<
SELECT d.schoolid, a.yearid, COUNT(*)
FROM halloffame AS a, people AS b, schools AS c, collegeplaying AS d 
WHERE a.playerid = b.playerid AND a.playerid = d.playerid AND c.schoolid = d.schoolid
AND a.inducted = 'Y' AND c.schoolstate = 'CA'
GROUP BY d.schoolid, a.yearid
ORDER BY a.yearid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable without_view


In [42]:
without_view

schoolid,yearid,count
ucla,1962,1
stmarysca,1971,1
camerri,1990,1
cafrecc,1992,1
usc,1992,1
calpoly,2002,4
sandiegost,2007,3
capasad,2008,1
sandiegost,2010,2
calavco,2011,1


In [32]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [33]:
grader.check("q1di")

q1di results: All test cases passed!

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [45]:
with_view_query_str = "SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca GROUP BY schoolid, yearid ORDER BY yearid"
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

                                                                            QUERY PLAN                                                                             
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=525.47..527.39 rows=96 width=20) (actual time=6.293..6.307 rows=13 loops=1)
   Group Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
   ->  Sort  (cost=525.47..525.71 rows=96 width=12) (actual time=6.285..6.291 rows=23 loops=1)
         Sort Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Subquery Scan on inducted_hof_ca  (cost=521.11..522.31 rows=96 width=12) (actual time=6.230..6.240 rows=23 loops=1)
               ->  Sort  (cost=521.11..521.35 rows=96 width=295) (actual time=6.228..6.234 rows=23 loops=1)
                     Sort Key: halloffame.yearid, people.playeri

In [46]:
without_view_query_str = "SELECT d.schoolid, a.yearid, COUNT(*) FROM halloffame AS a, people AS b, schools AS c, collegeplaying AS d WHERE a.playerid = b.playerid AND a.playerid = d.playerid AND c.schoolid = d.schoolid AND a.inducted = 'Y' AND c.schoolstate = 'CA' GROUP BY d.schoolid, a.yearid ORDER BY yearid;"
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

                                                                       QUERY PLAN                                                                        
---------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=521.11..523.03 rows=96 width=20) (actual time=5.793..5.807 rows=13 loops=1)
   Group Key: a.yearid, d.schoolid
   ->  Sort  (cost=521.11..521.35 rows=96 width=12) (actual time=5.786..5.791 rows=23 loops=1)
         Sort Key: a.yearid, d.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Nested Loop  (cost=386.71..517.95 rows=96 width=12) (actual time=5.262..5.729 rows=23 loops=1)
               Join Filter: ((a.playerid)::text = (b.playerid)::text)
               ->  Hash Join  (cost=386.42..485.79 rows=96 width=30) (actual time=5.226..5.600 rows=23 loops=1)
                     Hash Cond: ((a.playerid)::text = (d.playerid)::text)
             

In [34]:
with_view_cost = 527.39
with_view_timing = 6.534
without_view_cost = 523.03
without_view_timing = 6.007

In [35]:
grader.check("q1dii")

q1dii results: All test cases passed!

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [36]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS
SELECT namefirst, namelast, a.playerid, d.schoolid, a.yearid
FROM halloffame AS a, people AS b, schools AS c, collegeplaying AS d 
WHERE a.playerid = b.playerid AND a.playerid = d.playerid AND c.schoolid = d.schoolid 
AND a.inducted = 'Y' AND c.schoolstate = 'CA'
ORDER BY a.yearid, b.playerid;

SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
23 rows affected.
23 rows affected.
Returning data to local variable inducted_hof_ca_materialized


In [37]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [38]:
grader.check("q1ei")

q1ei results: All test cases passed!

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [39]:
%%sql with_materialized_view <<
SELECT schoolid, yearid, COUNT(*)
FROM inducted_hof_ca_mat
GROUP BY schoolid, yearid
ORDER BY yearid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_materialized_view


In [40]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [41]:
grader.check("q1eii")

q1eii results: All test cases passed!

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [55]:
with_materialized_view_query_str = "SELECT schoolid, yearid, COUNT(*) FROM inducted_hof_ca_mat GROUP BY schoolid, yearid ORDER BY yearid"
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=23.67..24.17 rows=200 width=60) (actual time=0.071..0.073 rows=13 loops=1)
   Sort Key: yearid
   Sort Method: quicksort  Memory: 25kB
   ->  HashAggregate  (cost=14.03..16.02 rows=200 width=60) (actual time=0.024..0.028 rows=13 loops=1)
         Group Key: yearid, schoolid
         Batches: 1  Memory Usage: 40kB
         ->  Seq Scan on inducted_hof_ca_mat  (cost=0.00..12.30 rows=230 width=52) (actual time=0.006..0.008 rows=23 loops=1)
 Planning Time: 0.390 ms
 Execution Time: 0.212 ms
(9 rows)



In [42]:
with_materialized_view_cost = 24.17
with_materialized_view_timing = 0.212

In [43]:
grader.check("q1eiii")

q1eiii results: All test cases passed!

#### Question 1eiv:
Given your findings from inspecting the query plans, as well as your understanding of views and materialized views from lectures, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [44]:
q1e_part4 = ['D','J']

In [45]:
grader.check("q1eiv")

q1eiv results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

In the case of virtual views, they are not saved and do not use physical memory so the cost to output all rows and the query execution time are very similar to querying the data from the tables without the view. The virtual view does not cause significant difference to execution time and cost or reduce the respective amounts. On the other hand, a materialized view results in better query performance in terms of execution time and cost signifantly reducing as the output is already stored on the disk and the data is precomputed. But when it comes to creating the actual view, materialized views take up certain storage space compared to no storage space from views so they take more time to be created than a regular view. 

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [136]:
query_view_str = "SELECT * FROM inducted_hof_ca"
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

                                                                       QUERY PLAN                                                                        
---------------------------------------------------------------------------------------------------------------------------------------------------------
 Subquery Scan on inducted_hof_ca  (cost=530.54..531.74 rows=96 width=33) (actual time=8.539..8.550 rows=23 loops=1)
   ->  Sort  (cost=530.54..530.78 rows=96 width=42) (actual time=8.537..8.542 rows=23 loops=1)
         Sort Key: a.yearid, a.playerid
         Sort Method: quicksort  Memory: 27kB
         ->  Nested Loop  (cost=386.71..527.38 rows=96 width=42) (actual time=7.533..8.454 rows=23 loops=1)
               Join Filter: ((a.playerid)::text = (b.playerid)::text)
               ->  Hash Join  (cost=386.42..485.79 rows=96 width=30) (actual time=7.386..7.907 rows=23 loops=1)
                     Hash Cond: ((a.playerid)::text = (d.playerid)::text)
                     ->  Seq S

In [46]:
query_view_cost = 530.78
query_view_timing = 8.770

In [47]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [48]:
%%sql result_2b <<
SELECT * FROM inducted_hof_ca WHERE yearid > 2010

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.
Returning data to local variable result_2b


In [49]:
result_2b

namefirst,namelast,playerid,schoolid,yearid
Pat,Gillick,gillipa99,calavco,2011
Pat,Gillick,gillipa99,usc,2011
Pat,Gillick,gillipa99,usc,2011
Randy,Johnson,johnsra05,usc,2015
Randy,Johnson,johnsra05,usc,2015
Randy,Johnson,johnsra05,usc,2015
Trevor,Hoffman,hoffmtr01,cacypre,2018
Trevor,Hoffman,hoffmtr01,cacypre,2018


In [141]:
query_view_with_filter_str = "SELECT * FROM inducted_hof_ca WHERE yearid > 2010"
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

                                                                                QUERY PLAN                                                                                 
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Subquery Scan on inducted_hof_ca  (cost=209.92..209.99 rows=6 width=33) (actual time=0.936..0.939 rows=8 loops=1)
   ->  Sort  (cost=209.92..209.93 rows=6 width=42) (actual time=0.935..0.936 rows=8 loops=1)
         Sort Key: a.yearid, a.playerid
         Sort Method: quicksort  Memory: 25kB
         ->  Nested Loop  (cost=0.85..209.84 rows=6 width=42) (actual time=0.597..0.892 rows=8 loops=1)
               Join Filter: ((a.playerid)::text = (b.playerid)::text)
               ->  Nested Loop  (cost=0.56..207.24 rows=6 width=30) (actual time=0.562..0.808 rows=8 loops=1)
                     ->  Nested Loop  (cost=0.29..190.63 rows=55 width=30) (ac

In [50]:
query_view_with_filter_cost = 209.93
query_view_with_filter_timing = 1.078

In [51]:
grader.check("q2b")

q2b results: All test cases passed!

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [52]:
q2c = ['A', 'E']

In [53]:
grader.check("q2c")

q2c results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

Filtering data to only include the observations that you need through a WHERE clause will dramatically improve query speed because the database can optimize the query using indexes and partitions which would reduce the amount of data and make operations faster in order to speed up acesss and thus reduce cost and execution time. 

<!-- END QUESTION -->



## Question 3: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

### Question 3a:
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [14]:
%%sql q3a_join <<
SELECT * 
FROM people INNER JOIN
collegeplaying ON people.playerid = collegeplaying.playerid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3a_join


Inspect the query plan for the query you wrote above.

In [15]:
q3a_query_str = "SELECT * FROM people INNER JOIN collegeplaying ON people.playerid = collegeplaying.playerid"
!psql -h localhost -d baseball -c "explain analyze $q3a_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..1193.88 rows=17350 width=167) (actual time=15.799..31.569 rows=17350 loops=1)
   Hash Cond: ((collegeplaying.playerid)::text = (people.playerid)::text)
   ->  Seq Scan on collegeplaying  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.016..2.238 rows=17350 loops=1)
   ->  Hash  (cost=619.70..619.70 rows=19370 width=146) (actual time=15.571..15.574 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.023..3.502 rows=19370 loops=1)
 Planning Time: 1.277 ms
 Execution Time: 32.627 ms
(8 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3a` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3a = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [54]:
q3a = ['C']

In [55]:
grader.check("q3a")

q3a results: All test cases passed!

### Question 3b

Similar to Question 3a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [18]:
%%sql q3b_join <<
SELECT * 
FROM people INNER JOIN
collegeplaying ON people.playerid = collegeplaying.playerid
ORDER BY people.playerid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3b_join


Inspect the query plan for the query you wrote above.

In [19]:
q3b_query_str = "SELECT * FROM people INNER JOIN collegeplaying ON people.playerid = collegeplaying.playerid ORDER BY people.playerid"
!psql -h localhost -d baseball -c "explain analyze $q3b_query_str"

                                                                        QUERY PLAN                                                                        
----------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..1910.20 rows=17350 width=167) (actual time=0.042..18.210 rows=17350 loops=1)
   Merge Cond: ((people.playerid)::text = (collegeplaying.playerid)::text)
   ->  Index Scan using master_pkey on people  (cost=0.29..1024.36 rows=19370 width=146) (actual time=0.017..4.296 rows=19368 loops=1)
   ->  Index Only Scan using collegeplaying_pkey on collegeplaying  (cost=0.29..620.54 rows=17350 width=21) (actual time=0.019..2.328 rows=17350 loops=1)
         Heap Fetches: 0
 Planning Time: 1.063 ms
 Execution Time: 18.955 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3b` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3b = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [56]:
q3b = ["B"]

In [57]:
grader.check("q3b")

q3b results: All test cases passed!

### Question 3c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [22]:
%%sql q3c_join <<
SELECT * 
FROM people AS a 
INNER JOIN
people AS b ON a.playerid != b.playerid
LIMIT 1000

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1000 rows affected.
Returning data to local variable q3c_join


Inspect the query plan for the query you wrote above.

In [23]:
q3c_query_str = "SELECT * FROM people AS a INNER JOIN people AS b ON a.playerid != b.playerid LIMIT 1000"
!psql -h localhost -d baseball -c "explain analyze $q3c_query_str"

                                                          QUERY PLAN                                                           
-------------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.00..15.00 rows=1000 width=292) (actual time=0.034..1.059 rows=1000 loops=1)
   ->  Nested Loop  (cost=0.00..5629241.33 rows=375177530 width=292) (actual time=0.033..0.988 rows=1000 loops=1)
         Join Filter: ((a.playerid)::text <> (b.playerid)::text)
         Rows Removed by Join Filter: 1
         ->  Seq Scan on people a  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.017..0.018 rows=1 loops=1)
         ->  Materialize  (cost=0.00..716.55 rows=19370 width=146) (actual time=0.008..0.516 rows=1001 loops=1)
               ->  Seq Scan on people b  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.002..0.126 rows=1001 loops=1)
 Planning Time: 0.888 ms
 Execution Time: 1.242 ms
(9 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3c` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [58]:
q3c = ['A']

In [59]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d
Please discuss your findings in the previous parts. In particular, we are interested in hearing why you think the query optimizer chooses the ultimate join approach in each of the above three scenarios. Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 3
-->

A hash join is used in the query processor for any query in which each join has an equi-join condition where there are no indexes that can be applied to the join conditions in order to sort the columns. 
A merge join is used in the case where the projections of the joined tables are already joined on the join columns as shown in 3b where the query output is ordered by playerid. They are faster and use less memory than hash joins as shown by the the lower execution time. 
A nested loop join is the simplest implementation of joining two tables and involves takeing a first value in the first table and comparing it to every value in the second inner table to detect a match and this makes sense for our query in 3c because we are trying to find pairs with different values. By limiting the output to 1000 entries in 3c, we wer able to reduce exeuction time and it reinforces how nested loops are great for joining tables where one is  small and the other is large and is considered the inner table that is indexed on the column used for the join. 

<!-- END QUESTION -->



## Question 4: Indexes

### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [60]:
%%sql result_4a <<
SELECT salaries.playerid, AVG(salary)
FROM salaries NATURAL JOIN appearances 
WHERE g_batting = 10
GROUP BY salaries.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
134 rows affected.
Returning data to local variable result_4a


In [20]:
result_4a

playerid,avg
anderla02,240000.0
ashbyan01,109000.0
ayraubo01,125000.0
bankswi01,240000.0
batchri01,157500.0
batiski01,100000.0
batismi01,153000.0
bautijo01,150000.0
beller01,145000.0
benesan01,68000.0


In [61]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [62]:
grader.check("q4ai")

q4ai results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [35]:
result_4a_query_str = "SELECT salaries.playerid, AVG(salary) FROM salaries NATURAL JOIN appearances WHERE g_batting = 10 GROUP BY salaries.playerid;"
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3638.34..3638.37 rows=1 width=17) (actual time=23.503..23.572 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=3638.34..3638.35 rows=1 width=17) (actual time=23.486..23.497 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2901.56..3638.33 rows=1 width=17) (actual time=14.127..23.282 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.te

In [63]:
result_4a_cost = 3638.37
result_4a_timing = 23.718

In [64]:
grader.check("q4aii")

q4aii results: All test cases passed!

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [23]:
%%sql result_4b <<
CREATE INDEX g_batting_idx ON appearances(g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4b


Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [46]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=2398.02..2398.04 rows=1 width=17) (actual time=12.347..12.426 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=2398.02..2398.02 rows=1 width=17) (actual time=12.330..12.343 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=1661.23..2398.01 rows=1 width=17) (actual time=3.423..12.130 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.tea

In [65]:
result_4b_cost = 2398.04
result_4b_timing = 12.603

In [66]:
grader.check("q4b")

q4b results: All test cases passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [24]:
%%sql 
DROP INDEX g_batting_idx 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [25]:
%%sql result_4c <<
CREATE INDEX salary_idx ON salaries(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4c


Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [51]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3638.34..3638.37 rows=1 width=17) (actual time=23.584..23.653 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=3638.34..3638.35 rows=1 width=17) (actual time=23.569..23.579 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2901.56..3638.33 rows=1 width=17) (actual time=14.571..23.381 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.te

In [67]:
result_4c_cost = 3638.37
result_4c_timing = 23.797

In [68]:
grader.check("q4c")

q4c results: All test cases passed!

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [69]:
q4d_part1 = ['D','E']

In [70]:
grader.check("q4di")

q4di results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

The original query retrieves the average salary for each player that batted in exactly 10 games and outputs both the playerid and average salary. If we we add an index on the g_batting column, it helps the query find the data quicker because we are already filtering the g_batting column and thus the execution time reduced to 12.603 and the cost dropped to 2398.04. 

For the salary index, it did not impact the query performance that resulted because it was still necessary to go through all salary values for each of the players that have batted in 10 games and the performance of calculating an average will always get slower the more records you have. Creating an index for the field that you want the average for isnt helpful because you dont want to do a lookup as you would want all the data as efficiently as possible

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [26]:
%%sql 
DROP INDEX salary_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [71]:
%%sql result_4ei <<
SELECT salaries.playerid, salaries.yearid, salaries.salary
FROM salaries NATURAL JOIN appearances 
WHERE g_batting = 10 and g_all = 10
GROUP BY salaries.playerid,salaries.yearid,salaries.salary;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
120 rows affected.
Returning data to local variable result_4ei


In [38]:
result_4ei

playerid,yearid,salary
anderla02,1986,240000.0
ashbyan01,1992,109000.0
ayraubo01,1993,125000.0
bankswi01,1995,240000.0
batchri01,1997,157500.0
batiski01,1991,100000.0
beller01,1993,145000.0
benesan01,1989,68000.0
bennega01,2008,825000.0
blackbu02,1994,3250000.0


In [72]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [73]:
grader.check("q4ei_part1")

q4ei_part1 results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [41]:
result_4ei_query_str = "SELECT salaries.playerid, salaries.yearid, salaries.salary FROM salaries NATURAL JOIN appearances WHERE g_batting = 10 and g_all = 10 GROUP BY salaries.playerid,salaries.yearid,salaries.salary;"
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                      QUERY PLAN                                                                                                       
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Group  (cost=3300.41..3300.43 rows=1 width=21) (actual time=36.505..36.555 rows=120 loops=1)
   Group Key: salaries.playerid, salaries.yearid, salaries.salary
   ->  Sort  (cost=3300.41..3300.42 rows=1 width=21) (actual time=36.500..36.512 rows=120 loops=1)
         Sort Key: salaries.playerid, salaries.yearid, salaries.salary
         Sort Method: quicksort  Memory: 34kB
         ->  Nested Loop  (cost=0.29..3300.40 rows=1 width=21) (actual time=20.955..36.171 rows=120 loops=1)
               ->  Seq Scan on appearances  (cost=0.00..3133.84 rows=21 width=20) (

In [74]:
result_4ei_cost = 3300.43
result_4ei_timing = 36.745

In [75]:
grader.check("q4ei_part2")

q4ei_part2 results: All test cases passed!

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [47]:
%%sql result_4eii << 
CREATE INDEX g_batting ON appearances(g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eii


In [48]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                      QUERY PLAN                                                                                                       
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Group  (cost=1782.41..1782.43 rows=1 width=21) (actual time=13.067..13.157 rows=120 loops=1)
   Group Key: salaries.playerid, salaries.yearid, salaries.salary
   ->  Sort  (cost=1782.41..1782.42 rows=1 width=21) (actual time=13.059..13.078 rows=120 loops=1)
         Sort Key: salaries.playerid, salaries.yearid, salaries.salary
         Sort Method: quicksort  Memory: 34kB
         ->  Nested Loop  (cost=18.62..1782.40 rows=1 width=21) (actual time=6.670..11.918 rows=120 loops=1)
               ->  Bitmap Heap Scan on appearances  (cost=18.33..1615.84 rows=21 wi

In [76]:
result_4eii_with_index_cost = 1782.43
result_4eii_with_index_timing = 13.357

In [77]:
grader.check("q4eii")

q4eii results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [51]:
%%sql 
DROP INDEX g_batting

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [78]:
%%sql result_4eiii <<
SELECT salaries.playerid, salaries.yearid, salaries.salary
FROM salaries NATURAL JOIN appearances 
WHERE g_batting = 10 OR g_all = 10
GROUP BY salaries.playerid,salaries.yearid,salaries.salary;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Returning data to local variable result_4eiii


In [79]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [80]:
grader.check("q4eiii_part1")

q4eiii_part1 results: All test cases passed!

In [17]:
result_4eiii_query_str = "SELECT salaries.playerid, salaries.yearid, salaries.salary FROM salaries NATURAL JOIN appearances WHERE g_batting = 10 OR g_all = 10 GROUP BY salaries.playerid,salaries.yearid,salaries.salary;"
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Group  (cost=3925.04..3925.06 rows=1 width=21) (actual time=53.248..53.358 rows=216 loops=1)
   Group Key: salaries.playerid, salaries.yearid, salaries.salary
   ->  Sort  (cost=3925.04..3925.05 rows=1 width=21) (actual time=53.244..53.275 rows=216 loops=1)
         Sort Key: salaries.playerid, salaries.yearid, salaries.salary
         Sort Method: quicksort  Memory: 41kB
         ->  Hash Join  (cost=3188.26..3925.03 rows=1 width=21) (actual time=36.297..52.690 rows=216 loops=1)
               Hash Cond

In [81]:
result_4eiii_cost = 3925.06
result_4eiii_timing = 53.574

In [82]:
grader.check("q4eiii_part2")

q4eiii_part2 results: All test cases passed!

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [20]:
%%sql result_4eiv << 
CREATE INDEX g_batting ON appearances(g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
(psycopg2.errors.DuplicateTable) relation "g_batting" already exists

[SQL: CREATE INDEX g_batting ON appearances(g_batting)]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [25]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Group  (cost=3925.04..3925.06 rows=1 width=21) (actual time=58.166..58.278 rows=216 loops=1)
   Group Key: salaries.playerid, salaries.yearid, salaries.salary
   ->  Sort  (cost=3925.04..3925.05 rows=1 width=21) (actual time=58.160..58.186 rows=216 loops=1)
         Sort Key: salaries.playerid, salaries.yearid, salaries.salary
         Sort Method: quicksort  Memory: 41kB
         ->  Hash Join  (cost=3188.26..3925.03 rows=1 width=21) (actual time=37.425..57.494 rows=216 loops=1)
               Hash Cond

In [83]:
result_4eiv_with_index_cost = 3928.68
result_4eiv_with_index_timing = 58.533

In [84]:
grader.check("q4eiv")

q4eiv results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [18]:
%%sql 
DROP INDEX g_batting

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [19]:
%%sql result_4v <<
CREATE INDEX g_batting_g_all_idx ON appearances(g_batting, g_all)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4v


In [20]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Group  (cost=3616.45..3616.46 rows=1 width=21) (actual time=34.257..34.351 rows=216 loops=1)
   Group Key: salaries.playerid, salaries.yearid, salaries.salary
   ->  Sort  (cost=3616.45..3616.46 rows=1 width=21) (actual time=34.253..34.276 rows=216 loops=1)
         Sort Key: salaries.playerid, salaries.yearid, salaries.salary
         Sort Method: quicksort  Memory: 41kB
         ->  Hash Join  (cost=2879.67..3616.44 rows=1 width=21) (actual time=11.538..33.674 rows=216 loops=1)
               Hash Cond

In [85]:
result_4ev_multiple_col_index_cost = 3616.46
result_4ev_multiple_col_index_timing = 35.686

In [86]:
grader.check("q4ev")

q4ev results: All test cases passed!

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [87]:
q4e_part6 = ['C', 'I']

In [88]:
grader.check("q4evi")

q4evi results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

When adding an index on a column used in the AND predicate, the execution time and query cost decreased to 1882.43 and 13.357. It retrieves the data for the g_batting column at a much faster rate which improves performance for the overall query as it allows for the short-circuiting property of AND to execute much faster where the indexed column any false value in the indexed column will allow for the other column to not be considered for that specific entry. 

The index created for the column used in the OR predicate did not reduce the query cost or timing because both columns in the OR predicate would need to be fully checked in order to satisfy the predicate and having an indexed column does not make a difference for the query performance. 

A multicolumn index are very useful when filtering by multiple columns which is the situation in this case and as a result both the cost and execution time decreased for the OR predicate query. 

<!-- END QUESTION -->



## Question 5: More Indexes
### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [25]:
%%sql result_5a_min << 
SELECT MIN(salary)
FROM salaries

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_min


In [26]:
result_5a_min_query_str = "SELECT MIN(salary) FROM salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=8.965..8.967 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.019..3.918 rows=26428 loops=1)
 Planning Time: 1.049 ms
 Execution Time: 9.091 ms
(4 rows)



In [89]:
result_5a_min_query_cost = 525.36
result_5a_min_query_timing = 9.091

In [90]:
grader.check("q5ai")

q5ai results: All test cases passed!

In [29]:
%%sql result_5a_avg <<
SELECT AVG(salary)
FROM salaries

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_avg


In [30]:
result_5a_avg_query_str = "SELECT AVG(salary) FROM salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=9.600..9.602 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.018..3.741 rows=26428 loops=1)
 Planning Time: 0.554 ms
 Execution Time: 9.713 ms
(4 rows)



In [91]:
result_5a_avg_query_cost = 525.36
result_5a_avg_query_timing = 9.713

In [92]:
grader.check("q5aii")

q5aii results: All test cases passed!

### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [42]:
%%sql 
CREATE INDEX salary_idx ON salaries(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [43]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                                  QUERY PLAN                                                                  
----------------------------------------------------------------------------------------------------------------------------------------------
 Result  (cost=0.32..0.33 rows=1 width=8) (actual time=0.093..0.094 rows=1 loops=1)
   InitPlan 1 (returns $0)
     ->  Limit  (cost=0.29..0.32 rows=1 width=8) (actual time=0.089..0.090 rows=1 loops=1)
           ->  Index Only Scan using salary_idx on salaries  (cost=0.29..762.78 rows=26428 width=8) (actual time=0.088..0.088 rows=1 loops=1)
                 Index Cond: (salary IS NOT NULL)
                 Heap Fetches: 0
 Planning Time: 1.018 ms
 Execution Time: 0.180 ms
(8 rows)



In [93]:
result_5b_min_query_cost = .33
result_5b_min_query_timing = .180

In [94]:
grader.check("q5bi")

q5bi results: All test cases passed!

In [46]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=13.433..13.435 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.020..4.541 rows=26428 loops=1)
 Planning Time: 0.801 ms
 Execution Time: 13.577 ms
(4 rows)



In [95]:
result_5b_avg_query_cost = 525.36
result_5b_avg_query_timing = 13.577

In [96]:
grader.check("q5bii")

q5bii results: All test cases passed!

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [49]:
%%sql 
DROP INDEX salary_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [97]:
q5c = ['D']

In [98]:
grader.check("q5c")

q5c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 5d:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

It is shown throuh the query plans that when an index is made on salary, finding the minimum and maxmum salary has a reduced exectution time (.180) and query cost (.33). The index essentially speeds up the query by allowing you to find a certain value in a table much easier which is exactly what the min() and max() functions do. On the other hand, COUNT() and AVG() still requieres the query to look throuh each entry in the table to calculate those values so an index would not help with optimization. 

<!-- END QUESTION -->



## Question 6: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [99]:
%%sql result_6a <<
SELECT playerid, yearid, teamid, ab
FROM batting
WHERE ab > 500

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8839 rows affected.
Returning data to local variable result_6a


In [100]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [101]:
grader.check("q6ai")

q6ai results: All test cases passed!

In [12]:
result_6a_query_str = "SELECT playerid, yearid, teamid, ab FROM batting WHERE ab > 500"
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2884.05 rows=8922 width=21) (actual time=0.288..13.647 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.408 ms
 Execution Time: 14.036 ms
(5 rows)



In [102]:
result_6a_cost = 2884.05
result_6a_timing = 14.036

In [103]:
grader.check("q6aii")

q6aii results: All test cases passed!

### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [15]:
%%sql 
CLUSTER batting USING batting_pkey

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [16]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2878.05 rows=8922 width=21) (actual time=0.016..14.241 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.477 ms
 Execution Time: 14.617 ms
(5 rows)



In [104]:
result_6b_cost = 1785.28
result_6b_timing = 14.617

In [105]:
grader.check("q6b")

q6b results: All test cases passed!

### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [19]:
%%sql 
CREATE INDEX ab_idx on batting(ab)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [20]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                      QUERY PLAN                                                      
----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on batting  (cost=101.44..1786.96 rows=8922 width=21) (actual time=0.862..5.263 rows=8839 loops=1)
   Recheck Cond: (ab > 500)
   Heap Blocks: exact=1311
   ->  Bitmap Index Scan on ab_idx  (cost=0.00..99.21 rows=8922 width=0) (actual time=0.660..0.661 rows=8839 loops=1)
         Index Cond: (ab > 500)
 Planning Time: 0.464 ms
 Execution Time: 5.700 ms
(7 rows)



In [106]:
result_6c_cost = 1786.96
result_6c_timing = 5.700

In [107]:
grader.check("q6c")

q6c results: All test cases passed!

### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [108]:
q6d = ['A','D', 'E', 'G']

In [109]:
grader.check("q6d")

q6d results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 6e:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

Creating a clustered index on the primary key and the ab column did decrease the total cost of the query in 6b to 1785.28 and decreased the total cost of the query in 6c to 18=786.96. The clustering on indexes in both cases creates a sorted data structure of the table rows according to their specified index key which results in a reduction in total cost when executing the query. However, in terms of execution time, the clustering based on the batting primary key did not improve execution time because the query is filtering on another column that is not the index or in this case primary key. For the ab_idx index, the query is filtering on a column that has a clustered index built on it which leads us to expect a faster execution time. 

<!-- END QUESTION -->



## Question 7: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [8]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [9]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 3.31 ms, sys: 2.16 ms, total: 5.47 ms
Wall time: 3.69 s


[]

In [110]:
result_7a_timing = 3.69

In [111]:
grader.check("q7a")

q7a results: All test cases passed!

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [12]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
300000 rows affected.


[]

### Question 7b:
Now, create an index on the `salary` column and record the **wall time** after executing the update script.

In [13]:
%%sql 
CREATE INDEX salary_idx ON SALARIES(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [14]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 1.82 ms, sys: 4.4 ms, total: 6.22 ms
Wall time: 5.38 s


[]

In [112]:
result_7b_timing = 5.38

In [113]:
grader.check("q7b")

q7b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 7c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened?

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

When adding an index onto the salaries table, the wall time increased from 3.69 s to 5.38 s upon retiming the update. This happened because it is hard to keep ongoing maintanance of the salary index when there is more and more data being added as the indexes will become fragmented and will perform slower. This maintance cost will increase the wall time whether there are updates, deletes, or inserts for the data.   

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 8: Takeaway

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

<!--
BEGIN QUESTION
name: q8
manual: true
points: 6
-->

1.One method that was emphasized with this project was indexing and its impact on optimizing queries. Indexing helps you look for data matching certain characteristics and when we use indexes on columns that we filter in queries the query execution time and cost is often reduces especially when the relations in the query are large. Indexes are generally great but it is important to use them carefully on selective attributes like unique or primary keys or on frequently queried data that is not regularly updated. 

2.Multiple column indexing is another method showcased in this project which involves storing data on more than just one column. They can achiee even greater decreases in query time and overall performance due to its ability to move through the data quicker as they are structured to have a hierarchical structure where each multicolumn index has a reference to each other. Overall, you can filter multiple columns for much fewer steps to obtain certain queries on large datasets where optimization is needed. 

3.Clustering a table based on an index is also a very important method that optimizes queries well and includes the rows in the data set being in the same order as the rows in the index. This allows for a more effective use of caching and results in less execution time and cost when using the index. It is especiallt useful for faster performance when the query is filtering on a column that has a clustered index that is built on it. 

<!-- END QUESTION -->



## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [114]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

  adding: results/ (stored 0%)
  adding: results/result_1a.csv (deflated 53%)
  adding: results/result_1c.csv (deflated 64%)
  adding: results/result_1di_view.csv (deflated 37%)
  adding: results/result_1di_no_view.csv (deflated 37%)
  adding: results/result_1ei.csv (deflated 64%)
  adding: results/result_1eii.csv (deflated 37%)
  adding: results/result_4a.csv (deflated 59%)
  adding: results/result_4ei.csv (deflated 62%)
  adding: results/result_4eiii.csv (deflated 64%)
  adding: results/result_6a.csv (deflated 72%)


/home/jovyan/fa22/proj/proj2/results.zip

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [115]:
grader.check_all()

q0 results: All test cases passed!

q1a results: All test cases passed!

q1bi results: All test cases passed!

q1bii results: All test cases passed!

q1c results: All test cases passed!

q1di results: All test cases passed!

q1dii results: All test cases passed!

q1ei results: All test cases passed!

q1eii results: All test cases passed!

q1eiii results: All test cases passed!

q1eiv results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q2c results: All test cases passed!

q3a results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q4ai results: All test cases passed!

q4aii results: All test cases passed!

q4b results: All test cases passed!

q4c results: All test cases passed!

q4di results: All test cases passed!

q4ei_part1 results: All test cases passed!

q4ei_part2 results: All test cases passed!

q4eii results: All test cases passed!

q4eiii_part1 results: All test cases passe

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()